# Step 3: Phase-folded transit plot

Goal: generate the essential visual check — a **phase-folded light curve** centered on the expected transit.

This step is intentionally "data-first": look at the phase-folded transit before interpreting any derived metrics.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

out = {
    'depth_ppm_hat': round(depth_ppm, 3),
    'depth_ppm_err': round(depth_err_ppm, 3),
    'radius_earth_est': round(sh.estimate_planet_radius_earth(depth_ppm=depth_ppm), 3),
}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_phase_folded
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE:
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='03_phase_folded')
    run_path = run_out_dir / 'phase_folded_transit.png'
    docs_path = (docs_out_dir / 'phase_folded_transit.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_phase_folded(lc, candidate, ax=ax)
    ax.set_title('TOI-5807.01: Phase-folded transit (stitched PDCSAP)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/03_phase_folded/phase_folded_transit.png`

![Phase-folded transit](../artifacts/tutorial_toi-5807-incremental/03_phase_folded/phase_folded_transit.png)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_ppm_err": 8.445,
  "depth_ppm_hat": 255.559,
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/03_phase_folded/phase_folded_transit.png",
  "radius_earth_est": 2.877,
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/03_phase_folded/phase_folded_transit.png"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **What you see:** a clear dip centered near phase 0 with depth ∼256 ppm (consistent with Step 2) and a stable out-of-transit baseline.
- **Why it’s useful:** this is the fastest reality check that we’re not chasing a phase-folding/ephemeris mistake.
- **Next step:** run the odd/even check (V01) and inspect the odd/even folded transit.

</details>
